In [1]:
import networkx as nx
from collections import defaultdict
from csv import DictReader, writer
import pandas as pd 
from matplotlib import cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import normalized_mutual_info_score, homogeneity_score
import random
import time
from igraph import *
import numpy as np
import graph_tool.all as gt
import csv
from datetime import datetime
from pathlib import Path

### import graph data

In [2]:
df = pd.read_csv("data/edgeData/edges_2013_1.csv")
df.drop(['pair_txt'], axis=1, inplace=True)
df["sum"] = df[["h0", "h1", "h2", "h3", "h4", "h5", "h6"]].sum(axis=1)
df.head()

node1                               node2  h0  \
0  14yNxGHpf4Buu3k4DmwfnvHsfTgbZNMyfz  1DHwxpAuDiAKYiQNep5UCMv6tL68urKB3L   2   
1  1DHwxpAuDiAKYiQNep5UCMv6tL68urKB3L   1QVpYsqzqSGoMu8QVhJC3f2HTFxBnGs1F   2   
2  1DHwxpAuDiAKYiQNep5UCMv6tL68urKB3L  1EEd5aTM29pPNqSCGn5kVrwiWAwSpLxjVR   2   
3  1DHwxpAuDiAKYiQNep5UCMv6tL68urKB3L  1GeehpFMUhprCbqWoDpE6Nd3vdFowpxCM2   2   
4  15JbSfZuCraDwoo7zwjL9r3WMP66W9SH1j  1DHwxpAuDiAKYiQNep5UCMv6tL68urKB3L   2   

   h1  h2  h3  h4  h5  h6  sum  
0   0   0   0   0   0   0    2  
1   0   0   0   0   0   0    2  
2   0   0   0   0   0   0    2  
3   0   0   0   0   0   0    2  
4   0   0   0   0   0   0    2

In [3]:
df.shape

(3183594, 10)

### import ground truth

In [4]:
start = time.time()
df_gt = pd.read_csv('data/ground_truth_id.csv.zip')
print(df_gt.shape)

df_gt = df_gt.dropna(axis=0, how='any', subset=["address", "entity"])
print(df_gt.shape)

df_gt = df_gt.drop_duplicates(subset="address", keep=False)
print(df_gt.shape)

entity_counts = df_gt["entity"].value_counts()
rare_entities = entity_counts[entity_counts < 10].index
df_gt = df_gt.loc[~df_gt["entity"].isin(rare_entities), :]
print(df_gt.shape)

df_gt.head()
print(time.time()-start)

(28027931, 4)
(28027931, 4)
(28027089, 4)
(28027060, 4)
115.05666613578796


In [5]:
gt_addr = set(df_gt["address"])
sample_addr = set(df["node1"]).union(set(df["node2"]))
known_addr = sample_addr.intersection(gt_addr)

print('ground truth addresses',len(gt_addr))
print('sample addressses:',len(sample_addr))
print('ground truth addresses in sample',len(known_addr))

ground truth addresses 28027060
sample addressses: 131252
ground truth addresses in sample 39161


In [6]:
df_gt_known = df_gt.loc[df_gt["address"].isin(sample_addr), ['address', 'entity']]

known_entities = set(df_gt_known['entity'])

# known_entity_addr_dict = {idx: set(df_gt_known.loc[df_gt_known["entity"] == e, "address"]) for idx, e in enumerate(known_entities)}

known_addr_entity_dict_list = [{a: idx for a in set(df_gt_known.loc[df_gt_known["entity"] == e, "address"])}
                               for idx, e in enumerate(known_entities)]
known_addr_entity_dict = {}
[known_addr_entity_dict.update(d) for d in known_addr_entity_dict_list]

known_entity_counts = df_gt_known['entity'].value_counts().rename("count").to_frame()

print('ground truth entities in sample:',len(known_entities))
known_entity_counts

ground truth entities in sample: 42


count
Instawallet.org        18268
CoinJoinMess            9604
SilkRoadMarketplace     4779
BTC-e.com               3923
Bitcoin.de               756
BitBargain.co.uk         356
BTCJam.com               305
BitZino.com              191
Bitstamp.net             187
Bitcoin-24.com           115
SealsWithClubs.eu         87
BTCC.com                  80
LocalBitcoins.com         76
Bitcash.cz                76
BitcoinFog                43
BitPay.com                36
CampBX.com                33
Bitmit.net                32
MercadoBitcoin.com.br     32
Cavirtex.com              31
BetsOfBitco.in            31
VirWoX.com                29
CoinURL.com               16
TheRockTrading.com        11
SatoshiDice.com            9
Bitfinex.com               8
BitNZ.com                  6
Cryptonit.net              6
Vircurex.com               6
OrderBook.net              5
FaucetBOX.com              4
McxNOW.com                 4
FYBSG.com                  3
Cryptsy.com                2
Just-Dice.com              2
BitMillions.com            2
Betcoins.net               2
BTCGuild.com               1
Bitcoin-Roulette.com       1
Cryptorush.in              1
MyBitcoin.com              1
BitAces.me                 1

In [7]:
def get_labels(cs_addr, known_addr_entity_dict):
    res = {}
    for idx, c in enumerate(cs_addr):
        for a in c:
            if a in known_addr:
                res[a] = idx
                
    labels_true = [] 
    labels_pred = []

    for a in res.keys():
        labels_true.append(known_addr_entity_dict[a])
        labels_pred.append(res[a])
    
    return labels_true, labels_pred

In [8]:
edge_tuples = df[["node1", "node2", "sum"]].itertuples(index=False)
g = Graph.TupleList(edge_tuples, directed=False, weights=True)
summary(g)

IGRAPH UNW- 131252 3183594 -- 
+ attr: name (v), weight (e)


In [9]:
df_res = pd.DataFrame({"degrees": g.degree(g.vs)})
now = datetime.now()
date_str = now.strftime("%Y%m%d_%H%M")
file_name_output = f"degree_dist_res_{date_str}.csv"
output_path = Path(f"data/res/{file_name_output}")
df_res.to_csv(output_path, index=False)

In [10]:
def get_initial_and_fixed(sample_size):
    known_idxs = [idx for idx, addr in enumerate(g.vs["name"]) if addr in known_addr]

    initial = np.zeros(len(g.vs), dtype=np.int)
    
    rng = np.random.RandomState(32512163)
    
    sample_known_idxs = rng.choice(known_idxs, size=sample_size, replace=False) # TODO: set seed
    non_sample_idxs = np.setdiff1d(range(len(g.vs)), sample_known_idxs)

    for idx in sample_known_idxs:
        initial[idx] = known_addr_entity_dict[g.vs[idx]["name"]]

    initial[non_sample_idxs] = range(sample_size, len(g.vs))
    
    fixed = np.zeros(len(g.vs), dtype=np.int)
    fixed[sample_known_idxs] = 1
    fixed = fixed.astype(bool)
    
    return initial, fixed

In [20]:
#Clusters
initial, fixed = get_initial_and_fixed(500)
lp = g.community_label_propagation(weights="weight", initial=initial, fixed=fixed)
print('number of clusters',len(lp))

#Cluster as dict
node_dict = {node:c_id for (c_id,cluster) in enumerate(lp) for node in cluster}
with open('data/gt_plots/node_clusters.csv','w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in node_dict.items():
        writer.writerow([key, value])
        
#Export edge list
edge_file = 'data/gt_plots/edges.csv'
g.write_edgelist(edge_file)

number of clusters 149


In [21]:
G = gt.load_graph_from_csv(edge_file, directed=False, csv_options={'delimiter': ' '})
cluster = G.new_vp(value_type='int', vals=node_dict.values())
G.vertex_properties["cluster"] = cluster
G.vp.cluster

<VertexPropertyMap object with value type 'int32_t', for Graph 0x7f2636fba550, at 0x7f2636fba4a8>

In [22]:
G.vp.cluster

<VertexPropertyMap object with value type 'int32_t', for Graph 0x7f2636fba550, at 0x7f2636fba4a8>

In [23]:
gt.label_components(G)[1]

array([131235,     17], dtype=uint64)

In [24]:
G

<Graph object, undirected, with 131252 vertices and 3183594 edges, 2 internal vertex properties, at 0x7f2636fba550>

### Plot

In [19]:
start = time.time()
loc1 = gt.sfdp_layout(G, groups=G.vp.cluster)
print("duration: {0:.2f} sec".format(time.time() - start))

start = time.time()
gt.graph_draw(G, edge_color='lightgrey', edge_pen_width=0.08, vertex_size=3,
             vertex_fill_color=G.vp.cluster, vcmap=cm.magma, 
             pos=loc1, output_size=(300,300), output="data/gt_plots/2013_2_sfdp_seed11_1.png");
print("duration: {0:.2f} sec".format(time.time() - start))

KeyboardInterrupt: 